In [1]:
!pip install tf-al

  Using cached tensorboard-2.2.2-py3-none-any.whl (3.0 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dropout, MaxPooling2D, Dense, Softmax

from tf_al import ExperimentSuitMetrics, Dataset, AcquisitionFunction, ExperimentSuitMetrics, Pool, Config
from tf_al.wrapper import McDropout
from tf_al.utils import setup_growth

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [4]:
inputs = np.expand_dims(np.vstack([x_train, x_test])/255., axis=-1)
targets = np.hstack([y_train, y_test])

In [5]:
print("Inputs-shape: ", inputs.shape)
print("Targets-shape:", targets.shape)

Inputs-shape:  (70000, 28, 28, 1)
Targets-shape: (70000,)


In [6]:
seed = 83152
np.random.seed(seed)
tf.random.set_seed(seed)

In [7]:
# Create dataset splits
val_set_size = 100
test_set_size = 10_000

x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=test_set_size)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=val_set_size)
init_indices = [836, 1034, 3278, 6563, 7656, 9477, 9510, 13228, 16375, 20170, 21573, 24311, 26170, 26971, 28686, 30281, 32193, 50056, 50696, 51906]

pool = Pool(x_train, y_train)
pool.annotate(init_indices)

In [8]:
pool.get_labeled_indices()

array([  836,  1034,  3278,  6563,  7656,  9477,  9510, 13228, 16375,
       20170, 21573, 24311, 26170, 26971, 28686, 30281, 32193, 50056,
       50696, 51906])

In [9]:
setup_growth()

1 Physical GPU's,  1 Logical GPU's


In [10]:
base_model = Sequential([
        Conv2D(32, 3, activation=tf.nn.relu, padding="same", input_shape=(28, 28, 1)),
        Conv2D(64, 3, activation=tf.nn.relu, padding="same"),
        MaxPooling2D(),
        Dropout(.25),
        Flatten(),
        Dense(128, activation=tf.nn.relu),
        Dropout(.5),
        Dense(10),
        Softmax()  
    ])

In [11]:
sample_size = 25
config = Config(
    fit = {"epochs": 200, "batch_size": sample_size},
    query = {"sample_size": sample_size},
    eval={"batch_size": 800, "sample_size": sample_size}
)

mc_model = McDropout(base_model, config=config)
mc_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [12]:
mc_model.evaluate(x_test, y_test, sample_size=sample_size, batch_size=900)

{'loss': 2.3054852, 'accuracy': 0.08986}

In [13]:
x_lab, y_lab = pool.get_labeled_data()
mc_model.fit(x_lab, y_lab)

Epoch 1/200
1/1 [==============================] - 0s 3ms/step - loss: 2.3212 - sparse_categorical_accuracy: 0.1000
Epoch 2/200
1/1 [==============================] - 0s 2ms/step - loss: 2.2348 - sparse_categorical_accuracy: 0.2000
Epoch 3/200
1/1 [==============================] - 0s 2ms/step - loss: 2.0682 - sparse_categorical_accuracy: 0.3500
Epoch 4/200
1/1 [==============================] - 0s 2ms/step - loss: 1.8993 - sparse_categorical_accuracy: 0.6000
Epoch 5/200
1/1 [==============================] - 0s 2ms/step - loss: 1.6517 - sparse_categorical_accuracy: 0.6500
Epoch 6/200
1/1 [==============================] - 0s 1ms/step - loss: 1.4744 - sparse_categorical_accuracy: 0.8000
Epoch 7/200
1/1 [==============================] - 0s 2ms/step - loss: 1.1188 - sparse_categorical_accuracy: 0.8500
Epoch 8/200
1/1 [==============================] - 0s 1ms/step - loss: 0.9952 - sparse_categorical_accuracy: 0.7000
Epoch 9/200
1/1 [==============================] - 0s 1ms/step - loss: 0

1/1 [==============================] - 0s 821us/step - loss: 4.8741e-04 - sparse_categorical_accuracy: 1.0000
Epoch 139/200
1/1 [==============================] - 0s 755us/step - loss: 0.0144 - sparse_categorical_accuracy: 1.0000
Epoch 140/200
1/1 [==============================] - 0s 696us/step - loss: 0.0024 - sparse_categorical_accuracy: 1.0000
Epoch 141/200
1/1 [==============================] - 0s 749us/step - loss: 7.3252e-04 - sparse_categorical_accuracy: 1.0000
Epoch 142/200
1/1 [==============================] - 0s 715us/step - loss: 0.0021 - sparse_categorical_accuracy: 1.0000
Epoch 143/200
1/1 [==============================] - 0s 822us/step - loss: 7.4163e-04 - sparse_categorical_accuracy: 1.0000
Epoch 144/200
1/1 [==============================] - 0s 739us/step - loss: 9.8072e-05 - sparse_categorical_accuracy: 1.0000
Epoch 145/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0026 - sparse_categorical_accuracy: 1.0000
Epoch 146/200
1/1 [=====================

In [14]:
mc_model.evaluate(x_test, y_test, batch_size=900, sample_size=sample_size)

{'loss': 1.8224133, 'accuracy': 0.52826}